## Notebook 03

### Passos
O primeiro passo é a importação dos pacotes necessários para a análise.

In [ ]:
import os, sys
import pickle
import re
from pathlib import Path

# load custom modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd

from pilotoeducacao.queries import put_results

# definir root como path
ROOT = Path().resolve().parent
DATA = ROOT / 'data' / 'rodada01'

rodada = 1

### Importar dados
Depois, nós devemos importar os resultados da busca para análise.

In [ ]:
# definir lista de resultados
resultados = []
arquivos = DATA.iterdir()
arquivos = sorted(list(arquivos))
arquivos = [arquivo for arquivo in arquivos if not re.search(r'csv', str(arquivo))]

for arquivo in arquivos:
    with open(arquivo, 'rb') as handle:
        resultado = pickle.load(handle)
        resultados.append(resultado)

### Análise Quantitativa
Primeiro nós demonstramos quantos resultados obtivémos usando os termos de busca definidos pelo nosso grupo. Na primeira busca, limitamos os resultados a 10.000 ocorrências para testarmos a qualidade do mecanismo de ranqueamento dos documentos do nosso servidor.

In [ ]:
termos = ['compras', 'educação', 'tecnologia']
dados = {
    'Exercício': ['Factual'] * 3 + ['Contrafactual'] * 3,
    'Termo': termos + termos[::-1], 
    'Hits': [r['hits']['value'] for r in resultados]
}
df = pd.DataFrame(data=dados)
df

### Cálculo de Matriz de Confusão
Para nosso cálculo de matriz de confusão, nós precisamos comparar os hits do exerício factual contra os hits do exercício contrafactual. Precisamos definir quatro grupos:
1. **G1 Factual Positivos:** grupo identificado pela busca sequencial compras >> educação >> tecnologia.
2. **G2 Factual Negativos:** grupo excluído pela busca sequencial compras >> educação >> tecnologia.
3. **G3 Contrafactual Positivos:** grupo identificado pela busca sequencial tecnologia >> educação >> compras.
4. **G4 Contrafactual Negativos:** grupo excluído pela busca sequencial tecnologia >> educação >> compras.

In [ ]:
# calcular os grupos
G1 = set(resultados[2]['ids'])
G2 = set(resultados[0]['ids']) - set(resultados[2]['ids'])
         
G3 = set(resultados[5]['ids'])
G4 = set(resultados[3]['ids']) - set(resultados[5]['ids'])

# definir grupos
true_positive = len(G1 & G3)
true_negative = len(G2 & G4)
false_positive = len(G1 - (G1 & G3))
false_negative = len(G2 - (G2 & G4))

print(f'Matriz de Confusão')
print(f'---------------------------')
print(f'True Positives (TP):  {true_positive:05d}')
print(f'True Negatives (TN):  {true_negative:05d}')
print(f'False Positives (FP): {false_positive:05d}')
print(f'False Negatives (FN): {false_negative:05d}')

##### Subir os Resultados na Planilha do Google
Finalmente, nós subimos os resultados para que tenhamos um registro das buscas de fácil acesso.

In [ ]:
google_creds = str(ROOT / 'tests/data/YOUR_CREDENTIALS.json')

# HITS
# range da planilha a substituir
range_ = f'queries!F{3*rodada-1}:J30'

# definir corpo dos valores a subir
value_range_body = {
    "majorDimension": "COLUMNS",
    'values': [
        [r['hits']['value'] for r in resultados[:3]], 
    ]
}

# imprimir resultados
put_results(google_creds, range_, value_range_body)

# MATRIZ DE CONFUSÃO
# range da planilha a substituir
range_ = f'resultados!B{1+rodada}:G2'

# definir corpo dos valores a subir
value_range_body = {
    "majorDimension": "ROWS",
    'values': [
        [
            true_positive,
            true_negative,
            false_positive,
            false_negative,
            round(true_positive/(true_positive+false_positive), 2),
            round(true_positive/(true_positive+false_negative), 2)
        ], 
    ]
}

# imprimir resultados
put_results(google_creds, range_, value_range_body)

### Análise Qualitativa
Para a análise qualitativa dos resultados, precisamos olhar para os scores do ES e sortear quais documentos analisar. Queremos, também, saber o tamanho dos documentos que estamos analizando, portanto, recuperamos também o tamanho de cada arquivo.

In [ ]:
# transformar resultados em banco de dados
dt = pd.json_normalize(resultados[0]['documents'])
dt[['sort.score', 'sort.id']] =  pd.DataFrame(dt['sort'].tolist(), index=dt.index)
dt.head()

# sortear e salvar em csv
dt.sample(n=5, random_state=42)

### Conclusão
Para repetirmos as análises, é só rodarmos o script `preparo_notebook.py` repetidamente selecionando o argumento da rodada de análise, e.g.

```python
python examples/preparo_notebook.py --rodada=2
```